* The following code is modified based on the original torchcv project. 
* We are going to use PASACAL VOC12 as dataset.
* You could donwload VOC2012 
  train/validation: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/#devkit
* test data: https://pjreddie.com/projects/pascal-voc-dataset-mirror/


In [1]:
import os
import cv2
import math
import random
import pickle
import argparse
import matplotlib.pyplot as plt
from PIL import Image
from time import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as T

from __future__ import print_function
from dbpn import Net as DBPN
from dbpn import get_pair_set
from ssd import SSD
from ssd import build_ssd
from ssd.layers.modules import MultiBoxLoss
from ssd.data.config import voc
from ssd.data import detection_collate
from ssd.data import VOCAnnotationTransform, VOCDetection, BaseTransform, SRDetection, DBPNLoader
from ssd.eval import test_net
from metric import *
from vgg import Vgg16

import sys; sys.argv=['']; del sys

# clean up device
torch.cuda.empty_cache()


In [2]:
# Global arguments & settings
parser = argparse.ArgumentParser(description='PyTorch Super Resolution Detection Networks')
parser.add_argument('--upscale_factor', type=int, default=4, help="super resolution upscale factor")
parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
parser.add_argument('--batch_size', type=int, default=2, help='training batch size') # GPU: 9GB!
parser.add_argument('--threads', type=int, default=2, help='number of threads for data loading')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')
parser.add_argument('--gpu_mode', type=bool, default=True)
parser.add_argument('--gpus', default=1, type=float, help='number of gpu')
#parser.add_argument('--test_dataset', type=str, default='VOC12-LR-X8-test')
#parser.add_argument('--sr_dataset', type=str, default='VOC12-SR-X8')
parser.add_argument('--train_dataset', type=str, default='VOC12-LR-x4')
parser.add_argument('--hr_dataset', type=str, default='VOC12-HR')
parser.add_argument('--anno_path', type=str, default='Annotations')
parser.add_argument('--imSetpath', type=str, default='ImageSets')
parser.add_argument('--input_dir', type=str, default='./dataset')
#parser.add_argument('--output', default='./dataset/results', help='Location to save some outputs')
parser.add_argument('--checkpt', default='./checkpoint', help='Location to save checkpoint models')
parser.add_argument('--lr', type=float, default=2e-5, help='Learning Rate. Default=0.00002')
parser.add_argument('--nEpochs', type=int, default=10, help='number of epochs to fine tune net S over target loss')
parser.add_argument('--ploss', type=bool, default=False,help='perceptual loss')

opt = parser.parse_args()

gpus_list=range(opt.gpus)
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)
if cuda:
    torch.cuda.manual_seed(opt.seed)
    torch.cuda.empty_cache()

Epochs = opt.nEpochs
tbz = opt.batch_size
num_classes = 21           # 20 (VOC0712) +1 for background



Namespace(anno_path='Annotations', batch_size=2, checkpt='./checkpoint', gpu_mode=True, gpus=1, hr_dataset='VOC12-HR', imSetpath='ImageSets', input_dir='./dataset', lr=5e-05, nEpochs=16, seed=123, testBatchSize=1, threads=2, train_dataset='VOC12-LR-x4', upscale_factor=4)


In [3]:
# Task validation Setup - We use VOC07 Detection validation for our SROD network

HOME = os.path.expanduser("~")
voc_root = os.path.join(HOME, "data/VOCdevkit/")

annopath = os.path.join(voc_root, 'VOC2007', 'Annotations', '%s.xml')
imgpath = os.path.join(voc_root, 'VOC2007', 'JPEGImages', '%s.jpg')
imgsetpath = os.path.join(voc_root, 'VOC2007', 'ImageSets', 'Main', '{:s}.txt')
YEAR = '2007'
devkit_path = voc_root + 'VOC' + YEAR
# VOC0712 image channel MEANS
dataset_mean = (104, 117, 123)

# Random picked detection network end-task validation set,
# N.B. it is not for SROD (DBPN+SSD) training target Loss.
set_type = '07val32-5702'
# set_typs is explained below:
"""
We have 5 validation sets to choose, size of 16, 32, 64, 128, 256 repectively,
Due to the time & computation constraint, we have these 5 downsampled val set.
They are randomly(no replacement) picked from  VOC2007/ImageSets/Main/val.txt.

Baseline mAP on Pre-trained ssd300 is measured as well:       (mAP)
These Detection baseline are measured on frozen SSD300 network using
'weights/ssd300_mAP_77.43_v2.pth', for 75x75 LR images been SR-ed by
VOC12 retrained DBPN baseline w. weight 'VOC12-LR-x4-DBPN-ep100.pth'.

~/data/VOCdevkit/VOC2007/ImageSets/Main/07val16-5648.txt      56.48%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val32-5702.txt      57.02%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val64-5826.txt      58.26%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val128-6019.txt     60.19%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val256-6030.txt     60.30%

These can be used to evaluate training in progress quantatively.
Again we do not run through whole val set for periodical check out, too slow!
"""

# Utility functions
def save_img(img, img_path):
    save_img = img.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
    cv2.imwrite(img_path, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])

# Dataloader for SuperVision Subnetwork Evaluation
eval_set = DBPNLoader(root='./dataset')
eval_loader = DataLoader(dataset=eval_set, batch_size=1, num_workers=1, shuffle=False)


# Network and Dataloader for Detection subnetwork to End Task Eval
# Tried to reuse 2nd half of our SRD net for detection, not lucky!
ssd300net = build_ssd('test', 300, num_classes)   # initialize SSD
ssd300net.load_state_dict(torch.load('ssd/weights/ssd300_mAP_77.43_v2.pth'))
ssd300net.eval()

print('Finished loading SSD300 task evaluation network!')
eval_det = VOCDetection(voc_root, [('2007', set_type)],
                       BaseTransform(300, dataset_mean),
                       VOCAnnotationTransform(), sr_path='./dataset/VOC07-SR-x4')
if cuda:
     ssd300net = ssd300net.cuda()
     cudnn.benchmark = True


Finished loading SSD300 task evaluation network!


/home/x/cs231n/super-resolution-detection/ssd/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/x/cs231n/super-resolution-detection/ssd/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [4]:
print('===> Loading SRD sub Net-S (DBPN) fine-tune training datasets (VOC12)')

sd_dataset = SRDetection(root='./dataset', image_sets='trainval.txt', data_mean = dataset_mean,
                         target_transform = VOCAnnotationTransform())

sd_data_loader = DataLoader(dataset=sd_dataset, batch_size=opt.batch_size, num_workers=opt.threads,
                            shuffle=False, collate_fn=detection_collate, pin_memory=True)


===> Loading SRD sub Net-S (DBPN) fine-tune training datasets (VOC12)


In [5]:
# TODO:
# 1. come up with test/eval functions
# 2. come up with visualization image

print('===> Architecture Definition of SRD network model: Net-S (DBPN) + Net-D (SSD300)')


class DBPN2SSD(nn.Module):
    
    def __init__(self, s_model_name, d_model_name, d_frozen):
        super(DBPN2SSD, self).__init__()
        self.supervis = DBPN(num_channels=3, base_filter=64, feat=256, num_stages=7, scale_factor=4)
        if os.path.exists(s_model_name):
            self.supervis = torch.nn.DataParallel(self.supervis, device_ids=gpus_list)
            self.supervis.load_state_dict(torch.load(s_model_name, map_location=lambda storage, loc: storage))

        # self.detector = SSD(), setup ssd as 'train' mode for gradient flow
        # later at test/eval situation, we will overwrite it's mode to 'test'
        self.detector = build_ssd('train', 300, num_classes)
        if os.path.exists(d_model_name):
            self.detector.load_state_dict(torch.load(d_model_name, map_location=lambda storage, loc: storage))
        if d_frozen:
            for param in self.detector.parameters():
                param.requires_grad = False
        
    def forward(self, x):
        superx = self.supervis(x)
        # current detector: SSD300, so assume superx: 300x300!
        detect = self.detector(superx)
        return (superx, detect)


#SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
#SDnet = DBPN2SSD('checkpoint/SRx4_4000.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
SRDnet = DBPN2SSD('dbpn/models/VOC12-LR-x4-DBPN-ep100.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)


===> Architecture Definition of SRD network model: Net-S (DBPN) + Net-D (SSD300)


/home/x/cs231n/super-resolution-detection/dbpn/dbpn.py:47: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/home/x/cs231n/super-resolution-detection/dbpn/dbpn.py:51: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/home/x/cs231n/super-resolution-detection/ssd/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/x/cs231n/super-resolution-detection/ssd/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [6]:
def mAP_eval():
    mAP = test_net('./eval', ssd300net, cuda, eval_det, BaseTransform(ssd300net.size, dataset_mean), \
                   5, 300, thresh=0.01, eval_set=set_type)
    return mAP
    
print(mAP_eval())


Evaluating detections
0.5384877025132182


In [7]:
# Main SRD training Loop

# Hyperparameters - balancing weights of net S loss vs. net D loss
# alpha = 1.0
# beta = 1.0

# load vgg network
dtype = torch.cuda.FloatTensor
vgg = Vgg16().type(dtype)
mse_loss = torch.nn.functional.mse_loss

Epochs = 16
# decay_steps = [5000, 10000, 20000, 40000, 80000] in iterations
# decay_steps = [5, 10, 20, 40, 80]              # in epochs
decay_steps = [17]                               # in epochs
gamma = 0.5
def adjust_learning_rate(optimizer, gamma=0.5, step=0):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = opt.lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

# discover 'best' dbpn model according to an eval set
best_dbpn_model = None
best_val = -1
prev_file = None


#################################################################
#
# Parameters:
#   alpha  - Loss weight from SuperV Net-S
#   beta   - Loss weight from Detect Net-D
#   pickle - Pickle file name for all data records
#
#################################################################
def train(SDnet, alpha = 1.0, beta = 1.0, pfname='a100b100.pkl'):
    best_val = -1
    prev_file = None
    pfix = pfname.split('/')[-1].split('.')[0]
    
    net = SDnet
    # current lr
    lrt = opt.lr
    
    # Dictionary to pickle all training/validation data
    trainval = {}
    
    # Following list are appended continously fr. Start2End
    # In unit sampling period of each validating invocation.
    # total running epochs
    epos = []
    # total running iterations vs. per epoch
    itrs = []
    # mean SROD training (compound) losses btw. validation
    mlos = []
    # mean APs,   sampled at each validation
    maps = []
    # mean PSNRs, sampled at each validation
    mpsn = []
    # mean SSIMs, sampled at each validation
    msim = []
    # current learning rate
    clr  = []
    

    # Criterions:
    #
    # net SR loss function, change to L2 later
    criterion_sr = nn.MSELoss()

    # ssd loss function
    criterion_ssd = MultiBoxLoss(voc['num_classes'], 0.5, True, 0, True, 3, 0.5, False, use_gpu=cuda)

    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        #net.supervis = torch.nn.DataParallel(net.supervis)
        cudnn.benchmark = True
        net = net.cuda(gpus_list[0])
        criterion_sr = criterion_sr.cuda(gpus_list[0])
        criterion_ssd = criterion_ssd.cuda(gpus_list[0])


    # optimizer = optim.SGD(net.supervis.parameters(), lr=opt.lr, \
    #                       momentum=opt.momentum, weight_decay=opt.weight_decay)
    #
    # Use universal Adam first :)
    # we can be Specific to what part of network's parameters to optimize, here is SuperR Net
    optimizer = optim.Adam(net.supervis.parameters(), lr=opt.lr, betas=(0.9, 0.999), eps=1e-8)
    
    # set model to training mode
    net.train()
    mAP_int = 0
    
    #max_iter = 20000
    max_iter  =  5600    # Approx. within 1 Epoch 5600/5770
    decay_step = 0
    for epoch in range(1, Epochs + 1):
        t_0 = time()
        epoch_loss = 0
        iters_loss = 0
        for iteration, batch in enumerate(sd_data_loader, 1):

            if iteration == max_iter + 1:
                break
            if epoch in decay_steps and iteration == 1:
                decay_step += 1
                lrt = adjust_learning_rate(optimizer, gamma, decay_step)

            # input is LR image; sr_target is pseudo (300x300) original HR image, det_target is bboxes
            input, sr_target, det_target = batch[0], batch[1], batch[2]
            if cuda:
                input = Variable(input.cuda(gpus_list[0]))
                sr_target = Variable(sr_target.cuda(gpus_list[0]))
                det_target = [Variable(ann.cuda(gpus_list[0]), volatile=True) for ann in det_target]
            else:
                input = Variable(input)
                sr_target = Variable(sr_target)
                det_target = [Variable(ann, volatile=True) for ann in det_target]

            optimizer.zero_grad()
            t0 = time()

            sr_out,ssd_out = net(input)
            
            if opt.ploss == False:
                loss_sr = criterion_sr(sr_out, sr_target)
            else:
                # Compute Perceptual Loss
                sr_hair_feature = vgg(sr_out)
                sr_feature = vgg(sr_target)
                loss_sr = mse_loss(sr_feature[1],sr_hair_feature[1])

            # Compound Loss from net-S: loss_sr and net-D: loss_l, loss_c
            loss_l, loss_c = criterion_ssd(ssd_out, det_target)
            loss = loss_sr * alpha + beta *(loss_l + loss_c)

            epoch_loss += loss.data[0]
            iters_loss += loss.data[0]
            loss.backward()
            optimizer.step()
            t1 = time()

            if iteration != 0 and iteration % 20 == 0:
                print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Time: {:.4f} sec.".format(epoch, \
                            iteration, len(sd_data_loader), loss.data[0], (t1 - t0)))

            # End Task (detection) & Imaging Quality Evaluation vs. Model Loss steps below:
            if iteration != 0 and iteration % 200 == 0:
                mItersLoss = iters_loss / 200
                iters_loss = 0
                net.eval()
                t2 = time()
                i, ssim, psnr  = 0, 0, 0
                for iter, bat in enumerate(eval_loader, 1):
                    # loader output:  lr,     hr,   fname
                    lr, hr, fname = bat[0], bat[1], bat[2]
                    # lr, hr, fname = eval_set.pull_item(i)
                    if cuda:
                        lr_v = Variable(lr.cuda(gpus_list[0]))
                    else:
                        lr_v = Variable(lr)
                    sr, _ = net(lr_v)
                    srimg = sr.cpu().data
                    hrimg = hr.data

                    # Struct Similarity Index & Peak Signal-Noise Ratio
                    ssim += metric_ssim(hrimg.numpy(), srimg.numpy())
                    psnr += metric_psnr(hrimg.numpy(), srimg.numpy())
                    i += 1
                    #print("ssim, psnr:", ssim, psnr)

                    image=srimg
                    image = (image - image.min())/(image.max() - image.min())
                    imagef = os.path.join('./dataset/VOC07-SR-x4/', '%s.jpg')
                    save_img(image, imagef % fname)

                mSSIM = ssim / i
                mPSNR = psnr / i
                mAP_acc = mAP_eval()
                t3 = time()
                print("Mean Loss = {:.4f}, Mean AP = {:.4f}, Mean SSIM = {:.4f}, Mean PSNR = {:.4f} || Time: {:.4f} sec."\
                      .format(mItersLoss, mAP_acc, mSSIM, mPSNR, (t3 - t2)))

                # Updating lists to be pickled
                epos.append(epoch)
                itrs.append(iteration + max_iter*(epoch-1))
                mlos.append(float(mItersLoss))
                maps.append(mAP_acc)
                mpsn.append(mPSNR)
                msim.append(mSSIM)
                clr.append(lrt)

                mAP_int = int(mAP_acc * 10000)     #  2318 =>  23.18 %
                mPSNR_i = int(mPSNR * 100)         #  2318 =>  23.18 dB
                if math.isnan(mSSIM):
                    mSSIM_i = 'NaN'
                else:
                    mSSIM_i = int(mSSIM * 10000)   #  2318 => 0.2318
                if mAP_int > best_val:
                    best_val = mAP_int
                    if prev_file is not None:
                        os.remove(prev_file)
                    chkfile = opt.checkpt + '/' + 'SRx4_e' + repr(epoch) + '_i' + repr(iteration) + '_mAP' + \
                              repr(best_val) + '_' + pfix + '_mPSNR' + repr(mPSNR_i) + '_mSSIM' + repr(mSSIM_i) + '.pth'
                    torch.save(net.supervis.state_dict(), chkfile)
                    prev_file = chkfile

                # back to training mode
                net.train()

            if iteration != 0 and iteration % 1000 == 0:
                print('Saving state, iter:', iteration)
                torch.save(net.supervis.state_dict(), opt.checkpt + '/' + 'SRx4_e' + repr(epoch) + \
                           '_i' + repr(iteration) + '_mAP' + repr(mAP_int) + '_' + pfix + '.pth')
        t_1 = time()
        print("===> Epoch {} Complete: Avg. Loss: {:.4f} || Time: {:.4f} sec."\
              .format(epoch, epoch_loss / (max_iter * tbz), (t_1 - t_0)))

    # pickle all the data records
    trainval['epochs'] = epos
    trainval['iters'] = itrs
    trainval['mloss'] = mlos
    trainval['m_aps'] = maps
    trainval['mpsnr'] = mpsn
    trainval['mssim'] = msim
    trainval['lrate'] = clr
    with open(pfname, 'wb') as f:
        pickle.dump(trainval, f)
        



In [8]:
# SROD Compund Loss relative ratio, Net-S vs. Net-D
# Alpha is weight Loss from Net-S (e.g. DBPN) Loss
# Beta  is weight Loss from Net-D (e.g. SSD ) Loss
AlphaBeta = [(1.00, 0.01),   (1.00, 0.10),   (1.00, 1.00),   (0.10, 1.00),   (0.01, 1.00)]
pickle_fs = ['a100b001.pkl', 'a100b010.pkl', 'a100b100.pkl', 'a010b100.pkl', 'a001b100.pkl']

# a, b = AlphaBeta[0]
# pf = pickle_fs[0]

a, b = AlphaBeta[1]
pf = pickle_fs[1]

train(SRDnet, a, b, pf)


__main__:115: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:131: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
__main__:132: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
__main__:138: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


===> Epoch[1](20/5770): Loss: 19962.4590 || Time: 0.3309 sec.
===> Epoch[1](40/5770): Loss: 2152.7407 || Time: 0.3231 sec.
===> Epoch[1](60/5770): Loss: 1744.4287 || Time: 0.3235 sec.
===> Epoch[1](80/5770): Loss: 1572.7965 || Time: 0.3256 sec.
===> Epoch[1](100/5770): Loss: 1161.6384 || Time: 0.3279 sec.
===> Epoch[1](120/5770): Loss: 854.1193 || Time: 0.3262 sec.
===> Epoch[1](140/5770): Loss: 1442.5859 || Time: 0.3271 sec.
===> Epoch[1](160/5770): Loss: 1123.3821 || Time: 0.3267 sec.
===> Epoch[1](180/5770): Loss: 1310.5184 || Time: 0.3265 sec.
===> Epoch[1](200/5770): Loss: 486.3425 || Time: 0.3352 sec.
Evaluating detections
Mean Loss = 3842.9856, Mean AP = 0.0735, Mean SSIM = 0.8028, Mean PSNR = 19.1769 || Time: 10.5015 sec.
===> Epoch[1](220/5770): Loss: 655.1283 || Time: 0.3279 sec.
===> Epoch[1](240/5770): Loss: 883.7861 || Time: 0.3284 sec.
===> Epoch[1](260/5770): Loss: 669.7334 || Time: 0.3284 sec.
===> Epoch[1](280/5770): Loss: 477.6986 || Time: 0.3274 sec.
===> Epoch[1](30

===> Epoch[1](2240/5770): Loss: 335.9167 || Time: 0.3285 sec.
===> Epoch[1](2260/5770): Loss: 206.7149 || Time: 0.3325 sec.
===> Epoch[1](2280/5770): Loss: 526.8494 || Time: 0.3562 sec.
===> Epoch[1](2300/5770): Loss: 396.3829 || Time: 0.3499 sec.
===> Epoch[1](2320/5770): Loss: 173.5297 || Time: 0.3741 sec.
===> Epoch[1](2340/5770): Loss: 657.5825 || Time: 0.3393 sec.
===> Epoch[1](2360/5770): Loss: 155.3345 || Time: 0.3434 sec.
===> Epoch[1](2380/5770): Loss: 436.9774 || Time: 0.3307 sec.
===> Epoch[1](2400/5770): Loss: 282.4861 || Time: 0.3487 sec.
Evaluating detections
Mean Loss = 369.8850, Mean AP = 0.4993, Mean SSIM = 0.9259, Mean PSNR = 23.3462 || Time: 9.9837 sec.
===> Epoch[1](2420/5770): Loss: 190.7076 || Time: 0.3320 sec.
===> Epoch[1](2440/5770): Loss: 623.2490 || Time: 0.3344 sec.
===> Epoch[1](2460/5770): Loss: 155.6670 || Time: 0.3334 sec.
===> Epoch[1](2480/5770): Loss: 517.0908 || Time: 0.3361 sec.
===> Epoch[1](2500/5770): Loss: 196.7940 || Time: 0.3366 sec.
===> Epoc

===> Epoch[1](4440/5770): Loss: 153.7265 || Time: 0.3560 sec.
===> Epoch[1](4460/5770): Loss: 97.0678 || Time: 0.3490 sec.
===> Epoch[1](4480/5770): Loss: 380.6863 || Time: 0.3475 sec.
===> Epoch[1](4500/5770): Loss: 275.9283 || Time: 0.3344 sec.
===> Epoch[1](4520/5770): Loss: 330.7005 || Time: 0.3380 sec.
===> Epoch[1](4540/5770): Loss: 605.9132 || Time: 0.3439 sec.
===> Epoch[1](4560/5770): Loss: 328.2525 || Time: 0.3455 sec.
===> Epoch[1](4580/5770): Loss: 301.2651 || Time: 0.3438 sec.
===> Epoch[1](4600/5770): Loss: 381.5839 || Time: 0.3513 sec.
Evaluating detections
Mean Loss = 306.1140, Mean AP = 0.4902, Mean SSIM = 0.9448, Mean PSNR = 23.9749 || Time: 10.1693 sec.
===> Epoch[1](4620/5770): Loss: 395.7127 || Time: 0.3345 sec.
===> Epoch[1](4640/5770): Loss: 422.4920 || Time: 0.3327 sec.
===> Epoch[1](4660/5770): Loss: 64.9264 || Time: 0.3425 sec.
===> Epoch[1](4680/5770): Loss: 138.6691 || Time: 0.3478 sec.
===> Epoch[1](4700/5770): Loss: 301.8798 || Time: 0.3405 sec.
===> Epoch

===> Epoch[2](1040/5770): Loss: 414.7585 || Time: 0.3384 sec.
===> Epoch[2](1060/5770): Loss: 181.1143 || Time: 0.3434 sec.
===> Epoch[2](1080/5770): Loss: 633.5015 || Time: 0.3327 sec.
===> Epoch[2](1100/5770): Loss: 407.1091 || Time: 0.3353 sec.
===> Epoch[2](1120/5770): Loss: 308.6705 || Time: 0.3407 sec.
===> Epoch[2](1140/5770): Loss: 221.6057 || Time: 0.3340 sec.
===> Epoch[2](1160/5770): Loss: 242.5862 || Time: 0.3310 sec.
===> Epoch[2](1180/5770): Loss: 226.7316 || Time: 0.3297 sec.
===> Epoch[2](1200/5770): Loss: 221.6826 || Time: 0.3360 sec.
Evaluating detections
Mean Loss = 315.0532, Mean AP = 0.5330, Mean SSIM = 0.9390, Mean PSNR = 23.5341 || Time: 10.0759 sec.
===> Epoch[2](1220/5770): Loss: 386.7786 || Time: 0.3297 sec.
===> Epoch[2](1240/5770): Loss: 362.7613 || Time: 0.3364 sec.
===> Epoch[2](1260/5770): Loss: 618.7888 || Time: 0.3362 sec.
===> Epoch[2](1280/5770): Loss: 379.2832 || Time: 0.3294 sec.
===> Epoch[2](1300/5770): Loss: 154.5715 || Time: 0.3346 sec.
===> Epo

...

===> Epoch[15](440/5770): Loss: 426.1846 || Time: 0.3344 sec.
===> Epoch[15](460/5770): Loss: 174.1560 || Time: 0.3339 sec.
===> Epoch[15](480/5770): Loss: 57.0778 || Time: 0.3421 sec.
===> Epoch[15](500/5770): Loss: 69.3553 || Time: 0.3399 sec.
===> Epoch[15](520/5770): Loss: 218.4218 || Time: 0.3352 sec.
===> Epoch[15](540/5770): Loss: 264.2701 || Time: 0.3326 sec.
===> Epoch[15](560/5770): Loss: 212.1113 || Time: 0.3310 sec.
===> Epoch[15](580/5770): Loss: 234.6054 || Time: 0.3315 sec.
===> Epoch[15](600/5770): Loss: 47.5993 || Time: 0.3325 sec.
Evaluating detections
Mean Loss = 208.7378, Mean AP = 0.5235, Mean SSIM = 0.9575, Mean PSNR = 25.4211 || Time: 10.0805 sec.
===> Epoch[15](620/5770): Loss: 114.7257 || Time: 0.3318 sec.
===> Epoch[15](640/5770): Loss: 201.1669 || Time: 0.3292 sec.
===> Epoch[15](660/5770): Loss: 115.2528 || Time: 0.3306 sec.
===> Epoch[15](680/5770): Loss: 187.1063 || Time: 0.3333 sec.
===> Epoch[15](700/5770): Loss: 263.7058 || Time: 0.3349 sec.
===> Epoch[

===> Epoch[15](2620/5770): Loss: 59.3585 || Time: 0.3294 sec.
===> Epoch[15](2640/5770): Loss: 180.7070 || Time: 0.3315 sec.
===> Epoch[15](2660/5770): Loss: 111.2372 || Time: 0.3318 sec.
===> Epoch[15](2680/5770): Loss: 334.0431 || Time: 0.3332 sec.
===> Epoch[15](2700/5770): Loss: 199.2713 || Time: 0.3522 sec.
===> Epoch[15](2720/5770): Loss: 295.3199 || Time: 0.3550 sec.
===> Epoch[15](2740/5770): Loss: 264.4098 || Time: 0.3331 sec.
===> Epoch[15](2760/5770): Loss: 92.6760 || Time: 0.3340 sec.
===> Epoch[15](2780/5770): Loss: 435.7123 || Time: 0.3410 sec.
===> Epoch[15](2800/5770): Loss: 134.7276 || Time: 0.3331 sec.
Evaluating detections
Mean Loss = 233.0872, Mean AP = 0.5307, Mean SSIM = 0.9569, Mean PSNR = 25.3899 || Time: 10.1108 sec.
===> Epoch[15](2820/5770): Loss: 217.5022 || Time: 0.3406 sec.
===> Epoch[15](2840/5770): Loss: 80.4234 || Time: 0.3362 sec.
===> Epoch[15](2860/5770): Loss: 147.1386 || Time: 0.3372 sec.
===> Epoch[15](2880/5770): Loss: 285.3478 || Time: 0.3573 se

Evaluating detections
Mean Loss = 210.9027, Mean AP = 0.5253, Mean SSIM = 0.9572, Mean PSNR = 25.3435 || Time: 10.0988 sec.
===> Epoch[15](4820/5770): Loss: 43.8692 || Time: 0.3306 sec.
===> Epoch[15](4840/5770): Loss: 287.9180 || Time: 0.3356 sec.
===> Epoch[15](4860/5770): Loss: 346.7438 || Time: 0.3330 sec.
===> Epoch[15](4880/5770): Loss: 442.4224 || Time: 0.3594 sec.
===> Epoch[15](4900/5770): Loss: 227.0101 || Time: 0.3340 sec.
===> Epoch[15](4920/5770): Loss: 98.2301 || Time: 0.3309 sec.
===> Epoch[15](4940/5770): Loss: 131.0231 || Time: 0.3385 sec.
===> Epoch[15](4960/5770): Loss: 192.8639 || Time: 0.3326 sec.
===> Epoch[15](4980/5770): Loss: 210.2131 || Time: 0.3379 sec.
===> Epoch[15](5000/5770): Loss: 138.6490 || Time: 0.3345 sec.
Evaluating detections
Mean Loss = 229.1604, Mean AP = 0.5722, Mean SSIM = 0.9572, Mean PSNR = 25.4034 || Time: 10.0975 sec.
Saving state, iter: 5000
===> Epoch[15](5020/5770): Loss: 283.4044 || Time: 0.3306 sec.
===> Epoch[15](5040/5770): Loss: 79.

===> Epoch[16](1380/5770): Loss: 136.3631 || Time: 0.3393 sec.
===> Epoch[16](1400/5770): Loss: 225.4879 || Time: 0.3308 sec.
Evaluating detections
Mean Loss = 228.4361, Mean AP = 0.5200, Mean SSIM = 0.9572, Mean PSNR = 25.4243 || Time: 10.0663 sec.
===> Epoch[16](1420/5770): Loss: 240.8923 || Time: 0.3314 sec.
===> Epoch[16](1440/5770): Loss: 227.6538 || Time: 0.3403 sec.
===> Epoch[16](1460/5770): Loss: 235.6743 || Time: 0.3319 sec.
===> Epoch[16](1480/5770): Loss: 184.1040 || Time: 0.3317 sec.
===> Epoch[16](1500/5770): Loss: 279.0223 || Time: 0.3345 sec.
===> Epoch[16](1520/5770): Loss: 161.8639 || Time: 0.3388 sec.
===> Epoch[16](1540/5770): Loss: 226.6246 || Time: 0.3381 sec.
===> Epoch[16](1560/5770): Loss: 306.5453 || Time: 0.3316 sec.
===> Epoch[16](1580/5770): Loss: 90.4074 || Time: 0.3438 sec.
===> Epoch[16](1600/5770): Loss: 63.3647 || Time: 0.3422 sec.
Evaluating detections
Mean Loss = 225.0808, Mean AP = 0.5195, Mean SSIM = 0.9570, Mean PSNR = 25.4004 || Time: 9.9902 sec.

===> Epoch[16](3540/5770): Loss: 52.2688 || Time: 0.3351 sec.
===> Epoch[16](3560/5770): Loss: 119.2441 || Time: 0.3319 sec.
===> Epoch[16](3580/5770): Loss: 185.9588 || Time: 0.3317 sec.
===> Epoch[16](3600/5770): Loss: 139.1834 || Time: 0.3304 sec.
Evaluating detections
Mean Loss = 205.1273, Mean AP = 0.5266, Mean SSIM = 0.9571, Mean PSNR = 25.3579 || Time: 10.0189 sec.
===> Epoch[16](3620/5770): Loss: 238.4521 || Time: 0.3281 sec.
===> Epoch[16](3640/5770): Loss: 151.7383 || Time: 0.3322 sec.
===> Epoch[16](3660/5770): Loss: 143.0640 || Time: 0.3321 sec.
===> Epoch[16](3680/5770): Loss: 367.6461 || Time: 0.3340 sec.
===> Epoch[16](3700/5770): Loss: 123.7847 || Time: 0.3360 sec.
===> Epoch[16](3720/5770): Loss: 506.9664 || Time: 0.3383 sec.
===> Epoch[16](3740/5770): Loss: 252.6246 || Time: 0.3303 sec.
===> Epoch[16](3760/5770): Loss: 106.4387 || Time: 0.3353 sec.
===> Epoch[16](3780/5770): Loss: 221.3364 || Time: 0.3465 sec.
===> Epoch[16](3800/5770): Loss: 51.5501 || Time: 0.3445 s

In [ ]:
torch.cuda.empty_cache()


### Debugging code below, no need to run :)

In [ ]:
# unpickle data records for visual
with open(pf, 'rb') as f:
    trainval = pickle.load(f)


In [ ]:
print(' YOU Donot need to run this ===> Loading some datasets')

lr_path = os.path.join(opt.input_dir, opt.train_dataset)
hr_path = os.path.join(opt.input_dir, opt.hr_dataset)

fine_train_set = get_pair_set(lr_path, hr_path)
train_data_loader = DataLoader(dataset=fine_train_set, num_workers=opt.threads, \
                               batch_size=opt.testBatchSize, shuffle=False)


In [ ]:
def simple_test():
    SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    net = SDnet
    
    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
        net = net.cuda()

    else:
        torch.set_default_tensor_type('torch.FloatTensor')


    
simple_test()

In [ ]:
# dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor ## UNCOMMENT THIS LINE IF YOU'RE ON A GPU!

# Real samples were all associated with Ture labels - 1
T_labels = torch.ones(logits_real.size()).type(dtype)
